In [1]:
import os
import dotenv
import psycopg2
from faker import Faker
from datetime import datetime
from sqlalchemy import create_engine, text
import pandas as pd
from models import samples, projects, sample_types, publications, projects_publications, sops, projects_sops, projects_samples, assays, studies, investigations, investigations_projects, projects_sample_types
from populate_db import generate_fake_data
from typing import Optional, Any

# load environment variables
dotenv.load_dotenv()
fake = Faker()

In [2]:
HOST = os.getenv('HOST')
USER = os.getenv('USER')
DBNAME = os.getenv('DBNAME')
PASSWORD = os.getenv('PASSWORD')

In [3]:
engine = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}/{DBNAME}')

In [4]:
# Fetch existing IDs
existing_ids = pd.read_sql('SELECT id FROM projects', engine)['id'].tolist()
existing_ids

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150]

In [5]:
# Fetch existing IDs
existing_ids = pd.read_sql('SELECT id FROM samples', engine)['id'].tolist()
existing_ids

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150]

In [6]:
query = """
SELECT 
    tables.table_name, 
    columns.column_name,
    columns.data_type,
    columns.character_maximum_length,
    columns.is_nullable,
    columns.column_default
FROM 
    information_schema.tables AS tables
JOIN 
    information_schema.columns AS columns
ON 
    tables.table_name = columns.table_name
WHERE 
    tables.table_schema = 'public'
ORDER BY 
    tables.table_name, 
    columns.ordinal_position;
    """
tables = pd.read_sql(query, engine)
tables


,table_name,column_name,data_type,character_maximum_length,is_nullable,column_default
0,assay_assets,id,integer,NaN,NO,nextval('assay_assets_id_seq'::regclass)
1,assay_assets,assay_id,integer,NaN,YES,None
2,assay_assets,asset_id,integer,NaN,YES,None
3,assay_assets,version,integer,NaN,YES,None
4,assay_assets,created_at,timestamp without time zone,NaN,YES,None
...,...,...,...,...,...,...
200,work_groups,name,character varying,255.0,YES,None
201,work_groups,institution_id,integer,NaN,YES,None
202,work_groups,project_id,integer,NaN,YES,None
203,work_groups,created_at,timestamp without time zone,NaN,YES,None


In [7]:
tables[tables["column_name"] == "uuid"]

,table_name,column_name,data_type,character_maximum_length,is_nullable,column_default
18,assays,uuid,character varying,255.0,YES,None
46,institutions,uuid,character varying,255.0,YES,None
53,investigations,uuid,character varying,255.0,YES,None
75,people,uuid,character varying,255.0,YES,None
91,projects,uuid,character varying,255.0,YES,None
116,publications,uuid,character varying,255.0,YES,None
131,sample_types,uuid,character varying,255.0,YES,None
145,samples,uuid,character varying,255.0,YES,None
163,sops,uuid,character varying,255.0,YES,None
177,studies,uuid,character varying,255.0,YES,None


In [8]:
id_cols = [i for i in tables[tables["data_type"] == "integer"]["column_name"].unique() if "id" in i]
id_cols

['id',
 'assay_id',
 'asset_id',
 'relationship_type_id',
 'study_id',
 'contributor_id',
 'assay_class_id',
 'policy_id',
 'suggested_assay_type_id',
 'suggested_technology_type_id',
 'sample_type_id',
 'assay_stream_id',
 'person_id',
 'work_group_id',
 'avatar_id',
 'project_id',
 'investigation_id',
 'status_id',
 'default_policy_id',
 'programme_id',
 'publication_id',
 'sample_id',
 'sop_id',
 'pubmed_id',
 'publication_type_id',
 'template_id',
 'originating_data_file_id',
 'institution_id']

In [9]:
# Example usage with the Project model
df = generate_fake_data(projects, num_rows=10)
df

,id,title,web_page,wiki_page,created_at,updated_at,description,avatar_id,default_policy_id,first_letter,site_credentials,site_root_uri,last_jerm_run,uuid,programme_id,default_license,use_default_policy,start_date,end_date
0,42,Husband under property section kitchen always ...,Player trouble eye he movie federal total. Cho...,Dream too himself level analysis. Likely even ...,2020-01-09 11:11:45.608873,2022-10-28 16:44:40.360526,Picture kind ten skill job wonder. Resource mi...,9,95,V,Day reflect soon throughout determine leader c...,Open knowledge remember single. Lay since secu...,2023-10-24 12:10:09.491996,Huge set thank. If situation offer whether tax...,61,Challenge million garden official. Memory adul...,False,2020-02-09,2022-09-07
1,64,Face recently wrong human consider about perso...,Card likely compare year both democratic. Work...,Few pay kid system. Key rest economy.\nVery pu...,2024-12-29 23:15:12.583738,2023-08-28 03:29:17.724652,Individual station cold evidence necessary. Wa...,9,43,S,Cover garden find good theory else. Number cou...,Population policy build town another degree so...,2020-12-10 08:30:04.490745,Establish structure herself last arrive.\nHe c...,82,Gas rest north care realize. Wait administrati...,False,2024-07-19,2024-04-10
2,85,Including movie least put nor.\nChallenge fina...,Trip company meet white store network. Off tho...,Space daughter learn space. Trouble pay unit c...,2024-06-22 18:29:04.788652,2020-03-27 16:52:39.822874,Who sport relationship toward. Loss fund Repub...,3,21,B,Factor people space anything policy from her p...,Reality white tonight thought particularly. Ca...,2022-03-10 22:13:17.404201,Against indicate may science share receive. Tr...,31,Mention concern try another across. Coach othe...,True,2020-12-06,2020-08-20
3,17,Size almost probably election left. Artist cou...,Friend amount ok culture. Any result fire clea...,Central weight director play. Meet mother step...,2024-02-23 12:45:02.019848,2021-02-24 19:15:43.681481,Perhaps enough tonight team defense. History f...,67,93,N,Piece world budget all let. Avoid similar do h...,Way while add message far.\nExplain follow sec...,2020-02-20 02:41:52.160500,Necessary sure myself way share training. Keep...,38,Never section law prevent. Thing end decade. S...,False,2025-01-13,2020-12-17
4,63,Represent loss to wife assume like quickly. Pe...,Chance exactly population why. Will individual...,Night task over evidence. Help they fund entir...,2020-05-14 21:12:50.699555,2020-06-20 19:13:20.206175,Nice inside lose.\nRaise grow rest. He money p...,68,67,W,Price similar world day father. Where ahead ra...,About movement certain. Evidence next scene bu...,2021-01-23 17:28:13.841074,Tv agree possible. Itself rich child daughter....,87,Behavior age score so. Without level left driv...,True,2023-10-31,2022-06-20
5,53,Result office religious evidence. Receive popu...,Again heavy type responsibility.\nBar too cons...,Law fear material page. We list carry.\nOthers...,2021-02-16 05:28:08.018989,2024-11-27 18:46:35.824397,Almost southern look without natural citizen m...,72,85,G,Eye state hotel subject very determine enough ...,Executive early individual rest authority pape...,2020-07-23 12:01:08.754022,Support radio institution fight table eye fire...,88,Decision north story themselves professor inst...,True,2020-03-27,2024-11-02
6,65,Place space everybody. Look customer old can d...,Money significant data by maintain get physica...,Determine least force try hit see. Protect ten...,2024-11-23 09:16:11.979785,2022-06-17 20:47:03.198220,Seat admit measure them person. Mission messag...,7,91,Y,Phone old what so political themselves glass a...,Hear expert together push subject. Case differ...,2023-01-23 19:58:42.586595,Long serve stay sport a thought. Service simpl...,27,Company over life really method religious smil...,True,2021-07-26,2021-02-20
7,44,Where he father. Create determine class partne...,Resource majority nor majority decision hair p...,Born m

In [10]:
# subset_tbls = ["samples", "projects", "sample_types", "publications", "projects_publications", "sops", "projects_sops", "projects_samples", "assays", "studies", "investigations", "investigations_projects", "projects_sample_types"]
schemas_dict = {
    "samples": samples,
    "projects": projects,
    "sample_types": sample_types,
    "publications": publications,
    "projects_publications": projects_publications,
    "sops": sops,
    "projects_sops": projects_sops,
    "projects_samples": projects_samples,
    "assays": assays,
    "studies": studies,
    "investigations": investigations,
    # "investigations_projects": investigations_projects,
    "projects_sample_types": projects_sample_types
}


In [11]:
# Start a transaction
with engine.begin() as connection:
    for table in schemas_dict.keys():
        # Use DELETE or TRUNCATE depending on your needs
        connection.execute(text(f'DELETE FROM {table};'))

In [12]:
tbl_dfs = {}
for tbl, schema in schemas_dict.items():
    tbl_dfs[tbl] = generate_fake_data(schema, num_rows=150)
tbl_dfs

{'samples':      id                                              title  sample_type_id  \
 0    52  The level American moment leg kitchen though. ...              87   
 1    24  Evening reduce many leave their peace project ...              64   
 2     3  Beautiful yeah debate admit know. Feeling read...               1   
 3    40  Between politics type. Bag also hospital someo...              52   
 4    77  Room town special finally wall near establish....              26   
 ..   ..                                                ...             ...   
 145  21  Bed hot source player hot sense generation. Ro...              95   
 146  12  Build certainly mission plant player just cent...              98   
 147  24  Maybe charge spring much. Produce material agr...              35   
 148   9  Organization trade trip look article unit boar...              54   
 149  25  Interesting candidate why rise. Her situation ...              87   
 
                                       

In [13]:
# Fetch existing IDs
# existing_ids = pd.read_sql('SELECT id FROM sample_types', engine)['id'].tolist()

# Generate unique IDs
def generate_unique_ids(num_rows, existing_ids):
    new_ids = set()
    while len(new_ids) < num_rows:
        new_id = fake.random_int(min=1, max=1000)
        if new_id not in existing_ids:
            new_ids.add(new_id)
    return list(new_ids)

In [14]:
def update_id_cols(df, id_cols, tbl):
    if any(col in df.columns for col in id_cols):
        cols = [col for col in id_cols if col in df.columns]
        for col in cols:
            existing_ids = pd.read_sql(f'SELECT {col} FROM {tbl}', engine)[col].tolist()
            df[col] = generate_unique_ids(num_rows=len(df), existing_ids=existing_ids)
    return df

In [15]:
# df = update_id_cols(tbl_dfs["sample_types"], id_cols, "sample_types")
# df.to_sql("sample_types", engine, if_exists='append', index=False)

In [16]:
for tbl, df in tbl_dfs.items():
    print(f"Updating {tbl}...")
    df = update_id_cols(df, id_cols, tbl)
    # Upload the DataFrame to the database
    df.to_sql(tbl, engine, if_exists='append', index=False)
    print(f"Successfully uploaded {tbl} to database")

Updating samples...
Successfully uploaded samples to database
Updating projects...
Successfully uploaded projects to database
Updating sample_types...


Successfully uploaded sample_types to database
Updating publications...
Successfully uploaded publications to database
Updating projects_publications...
Successfully uploaded projects_publications to database
Updating sops...
Successfully uploaded sops to database
Updating projects_sops...
Successfully uploaded projects_sops to database
Updating projects_samples...
Successfully uploaded projects_samples to database
Updating assays...
Successfully uploaded assays to database
Updating studies...
Successfully uploaded studies to database
Updating investigations...
Successfully uploaded investigations to database
Updating projects_sample_types...


IntegrityError: (psycopg2.errors.ForeignKeyViolation) insert or update on table "projects_sample_types" violates foreign key constraint "index_projects_sample_types_on_project_id"
DETAIL:  Key (project_id)=(4) is not present in table "projects".

[SQL: INSERT INTO projects_sample_types (project_id, sample_type_id) VALUES (%(project_id__0)s, %(sample_type_id__0)s), (%(project_id__1)s, %(sample_type_id__1)s), (%(project_id__2)s, %(sample_type_id__2)s), (%(project_id__3)s, %(sample_type_id__3)s), (%(p ... 6698 characters truncated ... 7)s), (%(project_id__148)s, %(sample_type_id__148)s), (%(project_id__149)s, %(sample_type_id__149)s)]
[parameters: {'project_id__0': 4, 'sample_type_id__0': 522, 'project_id__1': 517, 'sample_type_id__1': 15, 'project_id__2': 13, 'sample_type_id__2': 18, 'project_id__3': 14, 'sample_type_id__3': 20, 'project_id__4': 529, 'sample_type_id__4': 537, 'project_id__5': 17, 'sample_type_id__5': 538, 'project_id__6': 19, 'sample_type_id__6': 543, 'project_id__7': 53

In [ ]:
# conn = psycopg2.connect(
#     host=HOST,
#     dbname=DBNAME,
#     user=USER,
#     password=PASSWORD
# )
# cur = conn.cursor()